In [1]:
import torch
import os
import sys
import matplotlib
from matplotlib import pyplot as plt

sys.path.insert(1, "C:\\Users\\Maxim\\Documents\\GitHub\\Studium\\Bachelor\\schnetpack\\src")

import schnetpack as spk
import schnetpack.transform as trn

import numpy as np
import math
import tqdm

from ase.io.extxyz import read_xyz

In [2]:
model = torch.load("./jonas_hessian_500_loose")

with open("./found_conformers_run0.xyz", "r") as f:
    base_configuration = list(read_xyz(f, index = 0))[0]
    base_configuration2 = list(read_xyz(f, index = 1))[0]
    base_configuration3 = list(read_xyz(f, index = 2))[0]

In [4]:
%matplotlib qt

def plot_positions(atom):
    positions = atom.positions
    numbers = atom.numbers
    
    colors = {6: "black", 8: "red", 1: "blue"}
    colors = [colors[n] for n in numbers]
    charges = {6: "C", 8: "O", 1: "H"}
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # draw the bond between atom (0, 2) and (2, 8)
    for i, j in [(0, 2), (2, 8), (0, 1), (0, 3), (0, 4), (1, 7), (1, 5), (1, 6)]:
        x = [positions[i][0], positions[j][0]]
        y = [positions[i][1], positions[j][1]]
        z = [positions[i][2], positions[j][2]]
        ax.plot(x, y, z, c="gray")
        
    for i, (x, y, z) in enumerate(positions):
        ax.scatter(x, y, z, c=colors[i], label=i, s=100)
        ax.text(x, y, z + 0.15, f"{charges[numbers[i]]}", color='black')  # Shift the label slightly above the point
    
    
    # remove the axis
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    
    
    # rotate
    ax.view_init(elev=30, azim=20)
    
    plt.savefig("ethanol.png", dpi=300)
    
    # plt.legend()
    plt.show()

plot_positions(base_configuration)
# plot_positions(base_configuration2)
# plot_positions(base_configuration3)

In [5]:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
    
# rotate the connection between atom 0 and 2 (C-O bond)
# atom 8 the H atom is connected to atom 2 (O)
def rotate_C_O(atom, angle):
    # origin vector is the C-O bond
    origin = atom.positions[2] - atom.positions[0]
    # rotate the H atom around the C-O bond
    positions = atom.positions
    
    matrix = rotation_matrix(origin, math.radians(angle))
    new_H_position = np.dot(matrix, positions[8] - positions[2]) + positions[2]
    
    new_atom = atom.copy()
    new_atom.positions[8] = new_H_position
    
    return new_atom

def rotate_C_C(atom, angle):
    # origin vector is the C-C bond (atom 0 and 1)
    origin = atom.positions[1] - atom.positions[0]

    # rotate the H atoms (5, 6, 7) around the C-C bond
    positions = atom.positions
    
    matrix = rotation_matrix(origin, math.radians(angle))
    new_H5_position = np.dot(matrix, positions[5] - positions[1]) + positions[1]
    new_H6_position = np.dot(matrix, positions[6] - positions[1]) + positions[1]
    new_H7_position = np.dot(matrix, positions[7] - positions[1]) + positions[1]
    
    new_atom = atom.copy()
    new_atom.positions[5] = new_H5_position
    new_atom.positions[6] = new_H6_position
    new_atom.positions[7] = new_H7_position
    
    return new_atom
    

new_atom_1 = rotate_C_O(base_configuration, -120)
new_atom_2 = rotate_C_C(base_configuration, 60)

plot_positions(new_atom_1)
# plot_positions(new_atom_2)

In [6]:
converter = spk.interfaces.AtomsConverter(
    neighbor_list=trn.ASENeighborList(cutoff=5.0), dtype=torch.float32, device="cuda"
)

def get_energy(atom):
    inputs = converter(atom)
    out = model(inputs)
    return out["energy"].item()

energy = get_energy(base_configuration)
print(f"Energy of base configuration: {energy}")

Energy of base configuration: -97089.7899496979


In [7]:
energy1 = get_energy(base_configuration)
energy2 = get_energy(base_configuration2)
energy3 = get_energy(base_configuration3)

print(f"Energy of base configuration 1: {energy1}")
print(f"Energy of base configuration 2: {energy2}")
print(f"Energy of base configuration 3: {energy3}")

Energy of base configuration 1: -97089.7899458832
Energy of base configuration 2: -97090.16607026522
Energy of base configuration 3: -97090.16607598726


In [8]:
# Calculate potential energy surface
angles1 = np.linspace(-180, 180, 10)
angles2 = np.linspace(-180, 180, 10)

energies = []

for angle1 in tqdm.tqdm(angles1):
    for angle2 in angles2:
        atom = rotate_C_O(base_configuration, angle1)
        atom = rotate_C_C(atom, angle2)
        energy = get_energy(atom)
        energies.append((angle1, angle2, energy))



100%|██████████| 10/10 [01:05<00:00,  6.59s/it]


In [43]:
def plot_PES_3d(energies):
    x = np.array([p[0] for p in energies])
    y = np.array([p[1] for p in energies])
    z = np.array([p[2] for p in energies])

    # Create grid data for X and Y, using unique values
    x_unique = np.unique(x)
    y_unique = np.unique(y)
    X, Y = np.meshgrid(x_unique, y_unique)

    # Reshape Z data into the same shape as X and Y
    Z = np.array(z).reshape(len(x_unique), len(y_unique))

    # Create the figure and 3D axes
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot the surface
    surf = ax.plot_surface(X, Y, Z, cmap='viridis')

    # Labels and title
    ax.set_xlabel('Rotation C-O')
    ax.set_ylabel('Rotation C-C')
    ax.set_zlabel('Energy')
    ax.set_title('Potential Energy Surface (PES) - 3D Plot')
    
    ax.set_zticks([])

    # Show the plot
    plt.show()
    


def plot_PES_contour(energies):
    # Extract x, y, and z data from energies
    x = np.array([p[0] for p in energies])
    y = np.array([p[1] for p in energies])
    z = np.array([p[2] for p in energies])

    # Find the minimum energy at (0, 0)
    min_energy = min(z)

    # Calculate the energy difference relative to the minimum energy
    z_diff = np.array(z) - min_energy

    # Create grid data for X and Y, using unique values
    x_unique = np.unique(x)
    y_unique = np.unique(y)
    X, Y = np.meshgrid(x_unique, y_unique)

    # Reshape Z data into the same shape as X and Y
    Z_diff = z_diff.reshape(len(y_unique), len(x_unique))

    # Create the figure and contour plot
    fig, ax = plt.subplots(figsize=(8, 6))  # Adjusted size for tighter fit
    surf = ax.contourf(X, Y, Z_diff, levels=20, cmap='viridis', antialiased=False)
    for c in surf.collections:
        c.set_edgecolor("face")
        
    # Add color bar with scientific notation formatting
    cbar = fig.colorbar(surf, ax=ax, shrink=0.9)  # Shrink color bar
    cbar.set_label('ΔE to the equilibrium structure in kcal/mol', fontsize=18)
    cbar.ax.tick_params(labelsize=14)  # Font size for ticks

    # Labels and title with adjusted font size
    ax.set_xlabel('α', fontsize=18, labelpad=10)
    ax.set_ylabel('β', fontsize=18, labelpad=10)
    ax.set_title('Potential Energy Surface (PES) - Contour Plot', fontsize=18)

    # Adjust tick labels' font size
    ax.tick_params(axis='both', which='major', labelsize=18)

    # Set equal aspect ratio for better visualization
    ax.set_aspect('equal')

    plt.savefig("PES_contour.svg", dpi=300)
    # Show the plot
    plt.show()
    
# plot_PES_3d(energies)
plot_PES_contour(energies)


C:\Users\Maxim\AppData\Local\Temp\ipykernel_9480\2104212003.py:57: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed two minor releases later.
  for c in surf.collections:


In [35]:
energies_2d_CC = []
energies_2d_CO = []

angles = np.linspace(-180, 180, 90)
for angle in tqdm.tqdm(angles):
    atom = rotate_C_C(base_configuration, angle)
    energy = get_energy(atom)
    
    atom2 = rotate_C_O(base_configuration, angle)
    energy2 = get_energy(atom2)
    
    # energies_2d_CC.append((angle, energy))
    energies_2d_CO.append((angle, energy2))

100%|██████████| 90/90 [02:09<00:00,  1.44s/it]


In [37]:
def plot2d(energies):
    x = np.array([p[0] for p in energies])
    y = np.array([p[1] for p in energies])

    fig, ax = plt.subplots()
    ax.plot(x, y)
    ax.set_xlabel('Rotation C-O')
    ax.set_ylabel('Energy')
    ax.set_title('Potential Energy Surface')
    
    # set y lim
    ax.set_ylim(energy1 - 0.1, energy2 + 0.1)
    
    plt.show()
    
plot2d(energies_2d_CC)
plot2d(energies_2d_CO)

In [38]:
print(sorted(energies_2d_CC, key=lambda x: x[1]))

[(2.0224719101123583, -97089.77928761904), (-2.0224719101123583, -97089.77928571169), (-119.32584269662921, -97089.77084855501), (119.3258426966292, -97089.77084760134), (123.37078651685397, -97089.73738412325), (-123.37078651685394, -97089.73738126222), (115.28089887640448, -97089.71936444704), (-115.28089887640449, -97089.71935777132), (-6.067415730337075, -97089.69464520876), (6.067415730337075, -97089.69463948671), (127.41573033707868, -97089.62021569673), (-127.41573033707866, -97089.62020520632), (111.23595505617976, -97089.58508328859), (-111.23595505617978, -97089.58506993715), (10.112359550561791, -97089.528743071), (-10.112359550561791, -97089.52874211733), (131.4606741573034, -97089.42395619814), (-131.46067415730337, -97089.42395429079), (-107.19101123595506, -97089.37350587313), (107.19101123595505, -97089.3735030121), (14.157303370786508, -97089.28840188448), (-14.157303370786508, -97089.28839520876), (135.50561797752812, -97089.15657166902), (-135.5056179775281, -97089.1